![First Power Bi](.\images\dbachecks-logo.png )
# dbachecks

## Installation 

Take a look at the installation guidelines here [Intro](./01-dbachecks-Intro.ipynb)

Lets Check if you have the dbachecks module available

In [1]:
Get-Module dbachecks -ListAvailable

Directory: C:\Users\mrrob\Documents\PowerShell\Modules

ModuleType Version PreRelease Name PSEdition ExportedCommands

---------- ------- ---------- ---- --------- ----------------

Script 1.2.20 dbachecks Desk {Get-DbcConfig, Get-DbcConfigValue, Get-DbcReleaseNote, Set-DbcConfig…}

Script 1.2.15 dbachecks Desk {Get-DbcConfig, Get-DbcConfigValue, Get-DbcReleaseNote, Set-DbcConfig…}

...

Script 1.2.12 dbachecks Desk {Get-DbcConfig, Get-DbcConfigValue, Get-DbcReleaseNote, Set-DbcConfig…}

Directory: C:\Program Files\WindowsPowerShell\Modules

ModuleType Version PreRelease Name PSEdition ExportedCommands

---------- ------- ---------- ---- --------- ----------------

Script 1.2.15 dbachecks Desk {Get-DbcConfig, Get-DbcConfigValue, Get-DbcReleaseNote, Set-DbcConfig…}

## Examine the available commands

You can look at the commands in the dbachecks module

In [2]:
Get-Command -Module dbachecks

# All of the checks

You can see all of the checks using `Get-DbcCheck` 


In [3]:
Get-DbcCheck

Get-DbcCheck: The 'Get-DbcCheck' command was found in the module 'dbachecks', but the module could not be loaded. For more information, run 'Import-Module dbachecks'.

## Searching for Checks to run

You can search for checks using `Get-DbcCheck -Pattern SEARCHTERM`

In [12]:
Get-DbcCheck -Pattern backup

# Running a Check

This requires that you have the containers set up from the Notebook 00 - Setting up the containers for the rest of the containers.ipynb

If you have altered the folder path you will need to alter it below.

Running a check from the command line can be done with

`Invoke-DbcCheck -SqlInstance SQLINSTANCE -Check UNIQUETAG, A TAG, GROUP`

You can run multiple checks with

`Invoke-DbcCheck -SqlInstance SQLINSTANCE -Check Tag,AnotherTag,AnotherTag`

In [14]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstance = 'localhost,15592'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
$null = Reset-DbcConfig
Invoke-DbcCheck -SqlInstance $SqlInstance -SqlCredential $SqlCredential -Check AutoClose

## Auto-Complete for Checks
At the command line and in the editor the Checks will auto-complete

Place the cursor a space after the `-Check` and press CTRL + SPACE

`Invoke-DbcCheck -SqlInstance SQL2016N1 -Check `

In [9]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstance = 'localhost,15592'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstance -SqlCredential $SqlCredential -Check 

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Instance.Tests.ps1' with Tags DefaultTrace

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Instance.Tests.ps1

  Describing Default Trace

    Context Checking Default Trace on localhost,15592
      [+] The Default Trace should be enabled on localhost,15592 12ms
Tests completed in 755ms
Tests Passed: 1, Failed: 0, Skipped: 0, Pending: 0, Inconclusive: 0 


# Check Agent Jobs at the command line

Did all of the Agent Jobs succeed ?

In [1]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstance = 'localhost,15592'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstance -SqlCredential $SqlCredential -Check FailedJob

You may have no results because they are no enabled jobs that have run in the last 7 days (depending upon when I last updated the image). Lets run the jobs and see what we get

In [11]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstance = 'localhost,15592'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
(Get-DbaAgentJob -SqlInstance $SqlInstance -SqlCredential $SqlCredential).Start()
Start-Sleep -Seconds 15
Invoke-DbcCheck -SqlInstance $SqlInstance -SqlCredential $SqlCredential -Check FailedJob

## Check the instances are available
Are all of the instances up and available?

There will be failures for the default values of the authentication scheme as we are connecting with SQL Authentication.

In [3]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential -Check InstanceConnection 

# Check the databases are available
Are all the databases up available?

This by default will only show the system databases but we will talk about configuration for this in the next notebook

In [4]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  -Check DatabaseStatus

## Check that DBCC CheckDB 

This is one of my favourite checks to run. You can get the last time DBCC CHECKDB was run for a single database using T-SQL with

```sql
DBCC DBINFO('DatabaseName') WITH TABLERESULTS
```

You will find the result somewhere around line 50. Doing that for all of the databases on your estate is time consuming and prone to (human) error.
With dbachecks you can do it like this. ON my machine this checks the 2 containers and 14 databases to make sure DBCC CHECKDB was run in the last 7 days in under 2 seconds :-)

In [3]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  -Check LastGoodCheckDb

## Check that the Error log has no Errors

By default this will check the error logs for the last 24 hours

In [2]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  -Check ErrorLog

## Check the disk space on the instance

This will check that the disk space on the hosts has at least 20% free space by default. It requires passing the host names to the `-ComputerName` parameter

In [4]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -ComputerName $SqlInstances -Check DiskCapacity

# Check if DAC is enabled

This will check that DAC - the dedicated administrator connection is enabled (by default)

In [5]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  -Check DAC

## Check if the database collation matches the server collation

This will check that the collation of the database matches the collation of the instance to avoid collation conflict errors when using temporary tables

In [6]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  -Check DatabaseCollation

Are all of the SPNs correct ?

You should not run this one but look at the saved results to see what you can get.

In [6]:
$Instances =  'SQL2005Ser2003', 'SQL2008Ser12R2', 'SQL2012Ser08AG1', 'SQL2012Ser08AG2', 'SQL2012Ser08AG3', 'SQL2014Ser12R2', 'SQL2016N1', 'SQL2016N2', 'SQL2016N3', 'SQL2017N5', 'SQL2019N20', 'SQL2019N21', 'SQL2019N22', 'SQL2019N5'
Invoke-DbcCheck -SqlInstance $instances -Check SPN

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Server.Tests.ps1' with Tags SPN

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Server.Tests.ps1

  Describing SPNs

    Context Testing SPNs on SQL2005Ser2003
      [-] SQL2005Ser2003 should have a SPN Dont know the SPN for Dont know the Account 5ms
        Expected strings to be the same, because We expect to have a SPN Dont know the SPN for Dont know the Account, but they were different.
        Expected length: 4
        Actual length:   17
        Strings differ at index 0.
        Expected: 'None'
        But was:  'An Error occurred'
        191:     $SPN.Error | Should -Be 'None' -Because "We expect to have a SPN $($SPN.RequiredSPN) for $($SPN.InstanceServiceAccount)"
        at Assert-SPN, 

# Running multiple checks

All of the examples above have used a single unique tag. You can run more than one check at a time. You can do this either by specifying a tag that covers a number of checks. You can find these by running `Get-DbcCheck`

When you search for checks like disk

In [7]:
Get-DbcCheck disk

you can see that under AllTags for the DiskCapacity check there is a tag called Storage. If you search for storage

In [8]:
Get-DbcCheck Storage

you can see that this returns two checks - BackUpPathAccess and DiskCapacity. You can run both of these checks at the same time. Note that the `Type` of check is different for these two checks and you will have to supply both `SqlInstance` and `ComputerName`

In [9]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -ComputerName $SqlInstances -SqlCredential $SqlCredential -Check Storage

## Running checks for a Group

Checks are grouped into ... well groups! The groups are Database, Instance, Server, Agent, HADR, LogShipping and MaintenanceSolution. You can run all of the checks in a group by specifying the group. You will see failures and some errors due to missing configurations and because some checks wont run against containers

In [10]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential -Check Instance